In [1]:
## This script uses GPT-neo and Alphaca-lora to extract the text features
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")

# prepare input
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')

# forward pass
output = model(**encoded_input)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-04 15:58:37.388682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 15:58:38.192820: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-04 15:58:38.192891: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerr

In [12]:
import torch
from transformers import XLMRobertaTokenizer, XLMRobertaModel

# Load pre-trained XLM-RoBERTa tokenizer and model
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')
model = XLMRobertaModel.from_pretrained('xlm-roberta-large')

# Sample input text
input_text = "This is a sample input text for feature extraction."

# Tokenize input text
encoded_input = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt')

# Pass encoded input through the XLM-RoBERTa model
with torch.no_grad():
    features = model(**encoded_input)

sentence_features = features.last_hidden_state.mean(dim=1)
# Extract the final hidden states corresponding to each token
# last_hidden_states = features.last_hidden_state

# Average the last hidden states across all tokens to get sentence-level features
# sentence_features = torch.mean(last_hidden_states, dim=1)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
import random
import os
import re
import json
import numpy as np
import pandas as pd
# import lightgbm as lgbm
from datetime import datetime
from time import gmtime, strftime
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# import lightgbm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
import torch
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from transformers import ViltProcessor, ViltModel
# from transformers import ClipProcessor, ClipModel
import clip

from PIL import Image
device = "cuda" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-04 16:25:32.970157: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 16:25:33.884932: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-04 16:25:33.885012: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerr

In [2]:
all_data = pd.read_csv('../data/combine_data_530.csv', low_memory=False, index_col=0)
all_data = all_data.fillna('0')
all_data

,Category,Concept,Pid,Uid,Subcategory,Mediastatus,Pathalias,Ispublic,Alltags,Mediatype,...,ispro,user_description,canbuypro,timezone_timezone_id,photo_firstdate,timezone_offset,img,img_file,label,train_type
0,Fashion,glam,775,59@N75,Fashion,ready,luisdrayton,1,rock punk transgender tranny electronicmusic e...,photo,...,1,"0.0866962,-0.0752717,-0.0149407,-0.0460858,-0....",0,9,1213743830,1,train/59@N75/775.jpg,train/59@N75/775.jpg,11.18,0
1,Travel&Active&Sports,fifa,1075,1@N18,Soccer,ready,claudiolara,1,brazil rio brasil riodejaneiro by maria fifa m...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1@N18/1075.jpg,train/1@N18/1075.jpg,15.15,0
2,Entertainment,cinema,4890,351@N64,Movies,ready,rizzato,1,old cinema beauty marilyn photoshop movie joke...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,11,1296417600,-3,train/351@N64/4890.jpg,train/351@N64/4890.jpg,10.99,0
3,Holiday&Celebrations,old,6568,6@N59,Birthday,ready,4kleuren,1,pictures old family scans brothers sister 1958...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/6@N59/6568.jpg,train/6@N59/6568.jpg,8.63,0
4,Food,thirsty,7079,1617@N40,Drinks,ready,glosackmd,1,hot sahara animal animals desert bottles drink...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1617@N40/7079.jpg,train/1617@N40/7079.jpg,11.16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,Travel&Active&Sports,sea,1118324,49686@N77,Seattle,ready,larsgerrits,1,natur nature frog sea lake fish outdoor macro ...,photo,...,1,"0.150263,0.00344591,-0.158953,0.118596,-0.0566...",0,8,1394046082,0,test/49686@N77/1118324.jpg,test/49686@N77/1118324.jpg,-1.00,-1
486190,Travel&Active&Sports,cars,1118329,31147@N75,Cars,ready,None,1,11 arisk okruh veternov 2016 10 jn preov veter...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1460319331,0,test/31147@N75/1118329.jpg,test/31147@N75/1118329.jpg,-1.00,-1
486191,Animal,cat,1118336,5656@N26,Cats,ready,rogersg,1,holland netherlands amsterdam cat nl ramses gu...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,9,1464117797,1,test/5656@N26/1118336.jpg,test/5656@N26/1118336.jpg,-1.00,-1
486192,Whether&Season,white,1118349,28302@N36,Snowing,ready,ripensis,1,street people bw white black netherlands monoc...,photo,...,1,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1246915883,0,test/28302@N36/1118349.jpg,test/28302@N36/1118349.jpg,-1.00,-1


In [3]:
all_data['all_text'] = all_data['Title'] + all_data['Subcategory'] + all_data['Concept'] + all_data['Category'] + all_data['Alltags']
all_data

,Category,Concept,Pid,Uid,Subcategory,Mediastatus,Pathalias,Ispublic,Alltags,Mediatype,...,user_description,canbuypro,timezone_timezone_id,photo_firstdate,timezone_offset,img,img_file,label,train_type,all_text
0,Fashion,glam,775,59@N75,Fashion,ready,luisdrayton,1,rock punk transgender tranny electronicmusic e...,photo,...,"0.0866962,-0.0752717,-0.0149407,-0.0460858,-0....",0,9,1213743830,1,train/59@N75/775.jpg,train/59@N75/775.jpg,11.18,0,Luis Drayton - Edinburgh shoot #6FashionglamFa...
1,Travel&Active&Sports,fifa,1075,1@N18,Soccer,ready,claudiolara,1,brazil rio brasil riodejaneiro by maria fifa m...,photo,...,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1@N18/1075.jpg,train/1@N18/1075.jpg,15.15,0,Arena da Barra - Arena HSBC - Arena do PAN #...
2,Entertainment,cinema,4890,351@N64,Movies,ready,rizzato,1,old cinema beauty marilyn photoshop movie joke...,photo,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,11,1296417600,-3,train/351@N64/4890.jpg,train/351@N64/4890.jpg,10.99,0,MARILYN 2015MoviescinemaEntertainmentold cinem...
3,Holiday&Celebrations,old,6568,6@N59,Birthday,ready,4kleuren,1,pictures old family scans brothers sister 1958...,photo,...,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/6@N59/6568.jpg,train/6@N59/6568.jpg,8.63,0,Knikkertijd - 1959BirthdayoldHoliday&Celebrati...
4,Food,thirsty,7079,1617@N40,Drinks,ready,glosackmd,1,hot sahara animal animals desert bottles drink...,photo,...,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1617@N40/7079.jpg,train/1617@N40/7079.jpg,11.16,0,CAMELS01DrinksthirstyFoodhot sahara animal ani...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,Travel&Active&Sports,sea,1118324,49686@N77,Seattle,ready,larsgerrits,1,natur nature frog sea lake fish outdoor macro ...,photo,...,"0.150263,0.00344591,-0.158953,0.118596,-0.0566...",0,8,1394046082,0,test/49686@N77/1118324.jpg,test/49686@N77/1118324.jpg,-1.00,-1,Quaak!SeattleseaTravel&Active&Sportsnatur natu...
486190,Travel&Active&Sports,cars,1118329,31147@N75,Cars,ready,None,1,11 arisk okruh veternov 2016 10 jn preov veter...,photo,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1460319331,0,test/31147@N75/1118329.jpg,test/31147@N75/1118329.jpg,-1.00,-1,arisk okruh veternov 2016CarscarsTravel&Active...
486191,Animal,cat,1118336,5656@N26,Cats,ready,rogersg,1,holland netherlands amsterdam cat nl ramses gu...,photo,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,9,1464117797,1,test/5656@N26/1118336.jpg,test/5656@N26/1118336.jpg,-1.00,-1,Bike & Cat / Eerste Anjeliers DwarsstraatCatsc...
486192,Whether&Season,white,1118349,28302@N36,Snowing,ready,ripensis,1,street people bw white black netherlands monoc...,photo,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1246915883,0,test/28302@N36/1118349.jpg,test/28302@N36/1118349.jpg,-1.00,-1,The Stairs crewSnowingwhiteWhether&Seasonstree...


In [5]:
data = pd.DataFrame()
data['all_text'] = all_data['all_text']
data

,all_text
0,Luis Drayton - Edinburgh shoot #6FashionglamFa...
1,Arena da Barra - Arena HSBC - Arena do PAN #...
2,MARILYN 2015MoviescinemaEntertainmentold cinem...
3,Knikkertijd - 1959BirthdayoldHoliday&Celebrati...
4,CAMELS01DrinksthirstyFoodhot sahara animal ani...
...,...
486189,Quaak!SeattleseaTravel&Active&Sportsnatur natu...
486190,arisk okruh veternov 2016CarscarsTravel&Active...
486191,Bike & Cat / Eerste Anjeliers DwarsstraatCatsc...
486192,The Stairs crewSnowingwhiteWhether&Seasonstree...


In [6]:
all_data = ' '

In [12]:
import torch
import os
from transformers import XLMRobertaTokenizer, XLMRobertaModel
from tqdm import trange
os.environ['CUDA_VISIBLE_DEVICES'] = "4,5,6" 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the model and tokenizer
model = XLMRobertaModel.from_pretrained('xlm-roberta-large').to(device)
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large', truncation = True, max_length = 256, add_special_tokens = True, return_tensors="pt")
with torch.no_grad():
    text1 = data['all_text'][0]
    tokenizer1 = tokenizer(text1, truncation = True, max_length = 256, add_special_tokens = True, return_tensors="pt").to(device)
    outputs1 = model(**tokenizer1)
    xlm_roberta_fea = outputs1.last_hidden_state.mean(dim=1)
    for index in trange(486193):
        text = data['all_text'][index + 1]
        inputs = tokenizer(text, truncation = True, max_length = 256, add_special_tokens = True, return_tensors="pt").to(device)
        outputs = model(**inputs)
        xlm_roberta_fea1 =  outputs.last_hidden_state.mean(dim=1)
        xlm_roberta_fea = torch.concat([xlm_roberta_fea,xlm_roberta_fea1],0)
xlm_roberta_fea_1024 = pd.DataFrame(xlm_roberta_fea.cpu().detach().numpy())
xlm_roberta_fea_1024.columns = ['xlm_roberta_fea_{}'.format(i) for i in range(1024)]

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 23.69 GiB total capacity; 0 bytes already allocated; 941.19 MiB free; 0 bytes reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
xlm_roberta_fea_1024.to_csv('xlm_roberta_fea_1024.csv', header=True, index=None)

In [13]:
len(sentence_features)

1

In [14]:
aa = torch.cat([sentence_features,sentence_features],0)
aa

tensor([[ 0.0350, -0.0272,  0.0416,  ...,  0.0114, -0.0101, -0.0661],
        [ 0.0350, -0.0272,  0.0416,  ...,  0.0114, -0.0101, -0.0661]])

In [15]:
import pandas as pd
bb = pd.DataFrame(aa.detach().numpy())
bb

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.035026,-0.027155,0.04155,0.009068,0.094261,-0.055618,0.070645,0.246636,0.027395,-0.053654,...,0.066855,0.02103,0.005836,-0.067205,0.003695,0.016283,-0.061493,0.011381,-0.010115,-0.066078
1,0.035026,-0.027155,0.04155,0.009068,0.094261,-0.055618,0.070645,0.246636,0.027395,-0.053654,...,0.066855,0.02103,0.005836,-0.067205,0.003695,0.016283,-0.061493,0.011381,-0.010115,-0.066078


In [2]:
output

MaskedLMOutput(loss=None, logits=tensor([[[55.7384, -0.5452, 45.7036,  ..., 31.2872, 15.7994, 21.3703],
         [33.1532, -1.6156, 54.8843,  ..., 23.6705, 11.4019, 25.7337],
         [30.8617, -0.7453, 38.5328,  ..., 30.5612, 11.0940, 19.1339],
         ...,
         [35.4630, -1.7063, 51.9969,  ..., 28.3087, 13.9789, 28.7244],
         [50.2311, -1.0334, 56.6684,  ..., 39.6290, 19.2304, 27.5253],
         [34.1169, -0.5355, 56.7177,  ..., 29.0719, 13.3234, 20.8348]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [6]:
len(output[0])

1

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load the pretrained GPT model and tokenizer
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Define the input text
input_text = "Hello, how are you?"

# Tokenize the input text
tokenized_text = tokenizer.encode(input_text, add_special_tokens=True)

# Convert tokenized text to PyTorch tensors
input_ids = torch.tensor([tokenized_text])

# Generate embeddings
with torch.no_grad():
    outputs = model(input_ids)
    embeddings = outputs.last_hidden_state.mean(dim=1)

# Print the embeddings
print(embeddings)

ConnectionError: HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out.